Caso de Estudio
¿Cuándo es más eficiente usar concurrencia con múltiples Procesos, comparado con Múltiples Hilos?

Busca un ejemplo, ya sea en internet o uno que propongas, en donde sea más eficiente y explícalo en tus propias palabras.

Formato de entrega:

Notebook, donde des la explicación por escrito (descárgalo como PDF) ó
Video, donde expliques, junto con código, por qué es más eficiente.
Se evaluará la claridad con la que logres explicar por qué, para el ejemplo que propones, es más eficiente usar múltiples procesos que múltiples hilos. Pueden haber puntos extra por creatividad.

Si no logras que el multiproceso sea más rápido que el multihilo, explica pór qué crees que pasó.


In [ ]:
# === mp_groups.py ===
from concurrent.futures import ProcessPoolExecutor, as_completed
import multiprocessing as mp
import pandas as pd

# IMPORTA tu función tal cual la definiste en el notebook:
# from tu_modulo import train_catboost_region_tunueado_entrenar_evaluar

def _fit_one_group(args):
    """Función *top-level* (pickleable) para un grupo."""
    group_name, df_g, base_params, target, id_col = args
    # Ajuste importante: reducir threads internos de CatBoost dentro de cada proceso
    cat_params = {"thread_count": 2}   # ajústalo según tus cores
    model, metrics, scored = train_catboost_region_tunueado_entrenar_evaluar(
        df_g, target=target, base=base_params, cat_params=cat_params, id_col=id_col
    )
    return group_name, metrics, scored

if __name__ == "__main__":
    # 0) Carga tu DF completo (tal como lo haces en el notebook)
    full: pd.DataFrame = pd.read_parquet("../../notebooks/06_cb_regression/completo/tp_full.parquet")

    target = "IBD"
    id_col = "SamplingOperations_code"
    base_params = {
        # tus hiperparámetros base… (los mismos que ya usas)
        # ejemplo:
        # "loss_function":"RMSE","eval_metric":"RMSE","depth":7,"learning_rate":0.03,
        # "iterations":8000,"l2_leaf_reg":10,"min_data_in_leaf":32, ...
    }

    # 1) Partimos por grupo (cambia "Region" por la columna que tengas)
    if "Region" not in full.columns:
        raise KeyError("No encuentro la columna 'Region' para agrupar. Cámbiala por tu partición real.")

    tasks = []
    for region, df_g in full.groupby("Region", dropna=False):
        # Sugerencia: pasa solo el *slice* del grupo, no el DF completo
        tasks.append((region, df_g.copy(), base_params, target, id_col))

    # 2) Tamaño del pool: núcleos físicos // threads internos de CatBoost
    #    Ej. si tienes 8 cores y thread_count=2, usa 4 workers
    max_workers = max(1, mp.cpu_count() // 2)

    results = []
    with ProcessPoolExecutor(max_workers=max_workers) as ex:
        futures = [ex.submit(_fit_one_group, t) for t in tasks]
        for fut in as_completed(futures):
            results.append(fut.result())

    # 3) Recolecta resultados
    #    - metrics: lista/dict por región
    #    - scored: concat de predicciones sin label
    metrics_por_region = {region: m for region, m, _ in results}
    scored_concat = pd.concat([s for _, _, s in results], ignore_index=True)

    # 4) Guarda si quieres
    pd.DataFrame(metrics_por_region).T.to_csv("metrics_por_region.csv", index=True)
    scored_concat.to_csv("pred_sin_label_por_region.csv", index=False)


En el notebook que tengo adjunto al PDF del proyecto de Datatone, incluyo una función que diseñé para entrenar múltiples modelos de CatBoost de forma secuencial. Sin embargo, explico que este proceso puede tardar alrededor de 40 minutos en completarse cuando se ejecuta de manera lineal, ya que cada modelo se entrena uno por uno utilizando un solo núcleo del procesador. Por eso estoy compartiendo estos resultados junto con la propuesta de adaptar el código a multiprocessing, que permite que cada modelo se entrene en un núcleo distinto. De esta forma, el entrenamiento de todos los modelos puede ejecutarse de forma paralela y reducir significativamente el tiempo total de ejecución.

Es importante mencionar que al usar multiprocessing reduzco el thread_count dentro de cada modelo de CatBoost, porque al repartir los procesos entre núcleos ya no tiene sentido que cada modelo intente usar todos los hilos disponibles. En cambio, cada proceso puede aprovechar mejor su propio núcleo, y el scheduler del sistema operativo se encarga de distribuirlos dinámicamente entre los recursos físicos. Además, multiprocessing permite trabajar con rebanadas del DataFrame por grupo o por subconjunto de datos, de modo que cada proceso se enfoca en una porción distinta del problema y evita sobrecargar la memoria compartida o duplicar objetos muy grandes en los argumentos.

También destaco que CatBoost ya paraleliza internamente la construcción de árboles, pero multiprocessing va un paso más allá, porque convierte cada modelo en una ejecución completamente independiente. En otras palabras, mientras CatBoost distribuye tareas dentro de un mismo modelo, multiprocessing distribuye varios modelos distintos entre los núcleos disponibles. Esto no solo mejora la velocidad, sino que también aumenta la estabilidad del entrenamiento, porque cada proceso corre de forma aislada y sin interferencias del GIL de Python. Por eso, en este contexto, multiprocessing representa una mejora sustancial frente a la ejecución tradicional o incluso al multithreading, ya que aprovecha de forma mucho más eficiente la capacidad real del procesador.